# Importing libraries

In [56]:
# General libraries
import pandas as pd  
import numpy as np  
import cv2        
import os
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torchvision
from torchvision import transforms, datasets, models, utils
from torch.utils.data import Dataset, DataLoader 
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.nn import functional as F
from skimage import io, transform
from torch.optim import lr_scheduler
from skimage.transform import AffineTransform, warp

# Loading data using Dataset and DataLoader

In [57]:
class MyData(Dataset):
    def __init__(self, path_dir, test_ratio=0.2, train=True, transform=None):
        """
            Input:
                path_dir: train folder
                test_ratio: split size
            formart image_name: <number_id>_A<age>_G<0,1>.png
        """
        list_age = []
        list_gender = []
        list_path = []
        for image_name in os.listdir(path_dir):
            age = ((image_name.split(".")[0]).split("_")[1]).split("A")[-1]
            gender = ((image_name.split(".")[0]).split("_")[2]).split("G")[-1]
            image_path = os.path.join(path_dir, image_name)
            
            list_age.append(float(age))
            list_gender.append(int(gender))
            list_path.append(image_path)
        
        # max age
        self.max_age = max(list_age)

        # normalize age
        list_age = [age / self.max_age for age in list_age]

        # #Splitting the data into train and validation set
        X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = \
        train_test_split(list_path, list_age, list_gender, test_size=test_ratio)
        
        if train:
            self.X = X_train
            self.age_y = y_age_train
            self.gender_y = y_gender_train
        else:
            self.X = X_test
            self.age_y = y_age_test
            self.gender_y = y_gender_test
        
        # apply transformation
        self.transform=transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = cv2.imread(self.X[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image.astype('float')
        age = np.array(self.age_y[idx]).astype('float')
        gender = np.array(self.gender_y[idx]).astype('float')

        sample={'image': image, 'label_age': age, 'label_gender': gender}

        if self.transform:
            sample = self.transform(sample)
        
        return sample

In [58]:
class RGBToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, age, gender = sample['image'], sample['label_age'], sample['label_gender']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        image = torch.from_numpy(image).permute(2, 0, 1).float()
        age = torch.from_numpy(age).float()
        gender = torch.from_numpy(gender).float()

        return {'image': image,
                'label_age': age,
                'label_gender': gender}

In [59]:
transformed_train_data = MyData(path_dir="mega_age_gender", test_ratio=0.2, train=True, transform=transforms.Compose([RGBToTensor()]))
transformed_test_data = MyData(path_dir="mega_age_gender", test_ratio=0.2, train=False, transform=transforms.Compose([RGBToTensor()]))

train_dataloader = DataLoader(transformed_train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(transformed_test_data, batch_size=32, shuffle=True)

# Model

In [60]:
class AgeGenderModel(nn.Module):
    def __init__(self):
        super(AgeGenderModel, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=0.5)
        )

        self.fc_age = nn.Linear(73926, 1)  #For age class
        self.fc_gender = nn.Linear(73926, 1)    #For gender class
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)

        age = self.fc_age(x)
        gender= torch.sigmoid(self.fc_gender(x))  

        return {'age': age, 'gender': gender}

# device

In [61]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Setting params

In [62]:
#Setting model and moving to device
model_CNN = AgeGenderModel().to(device)
#For binary output:gender
criterion_binary= nn.BCELoss()
#For age output
criterion_regression = nn.MSELoss()
optimizer = optim.SGD(model_CNN.parameters(), lr=0.001, momentum=0.9)

# Train

In [65]:
def train_model(model, criterion_binary, criterion_regression, optimizer, n_epochs=25):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    for epoch in range(1, n_epochs):
        train_loss = 0.0
        train_loss_age = 0.0
        train_loss_gender = 0.0
        valid_loss = 0.0

        # train the model 
        model.train()
        for batch_idx, sample_batched in enumerate(train_dataloader):
            # importing data and moving to GPU
            image, label_age, label_gender = sample_batched['image'].to(device),\
                                             sample_batched['label_age'].to(device),\
                                              sample_batched['label_gender'].to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            output = model(image)
            label_age_hat = output['age']
            label2_gender_hat = output['gender']
     
            # calculate loss
            loss_age = criterion_regression(label_age_hat, label_age)
            loss_gender = criterion_binary(label2_gender_hat, label_gender.unsqueeze(-1))

      
            loss = loss_age + loss_gender
            # back prop
            loss.backward()
            # grad
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            train_loss_age = train_loss_age + ((1 / (batch_idx + 1)) * (loss_age.data - train_loss_age))
            train_loss_gender = train_loss_gender + ((1 / (batch_idx + 1)) * (loss_gender.data - train_loss_gender))

            if batch_idx % 50 == 0:
                print('Epoch %d, Batch %d, loss: %.6f loss_age: %.6f loss_gender: %.6f' %
                  (epoch, batch_idx + 1, train_loss, loss_age, loss_gender))
                
        # validate the model #
        model.eval()
        for batch_idx, sample_batched in enumerate(test_dataloader):
            image, label_age, label_gender = sample_batched['image'].to(device),\
                                             sample_batched['label_age'].to(device),\
                                              sample_batched['label_gender'].to(device)

            output = model(image)
            label_age_hat = output['age']
            label2_gender_hat = output['gender']
         
            # calculate loss
            loss_age = criterion_regression(label_age_hat, label_age)
            loss_gender = criterion_binary(label2_gender_hat, label_gender.unsqueeze(-1))


            loss = loss_age + loss_gender
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            torch.save(model, 'model.pt')
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [66]:
model_conv=train_model(model_CNN, criterion_binary, criterion_regression, optimizer, n_epochs=5)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Batch 1, loss: 0.740608 loss_age: 0.048498 loss_gender: 0.692110
Epoch 1, Batch 51, loss: 0.756001 loss_age: 0.055358 loss_gender: 0.695640
Epoch 1, Batch 101, loss: 0.753992 loss_age: 0.059784 loss_gender: 0.691849
Epoch 1, Batch 151, loss: 0.753774 loss_age: 0.084829 loss_gender: 0.698122
Epoch 1, Batch 201, loss: 0.754431 loss_age: 0.072938 loss_gender: 0.695431
Epoch 1, Batch 251, loss: 0.754496 loss_age: 0.058779 loss_gender: 0.695667
Epoch 1, Batch 301, loss: 0.754518 loss_age: 0.046270 loss_gender: 0.699324
Epoch 1, Batch 351, loss: 0.754341 loss_age: 0.069749 loss_gender: 0.689951
Epoch 1, Batch 401, loss: 0.754474 loss_age: 0.079494 loss_gender: 0.692130
Epoch 1, Batch 451, loss: 0.754727 loss_age: 0.069394 loss_gender: 0.693355
Epoch 1, Batch 501, loss: 0.754760 loss_age: 0.061668 loss_gender: 0.688476
Epoch 1, Batch 551, loss: 0.754833 loss_age: 0.055429 loss_gender: 0.691930
Epoch 1, Batch 601, loss: 0.754618 loss_age: 0.073693 loss_gender: 0.684038
Epoch 1, Batch 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([31])) that is different to the input size (torch.Size([31, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1 	Training Loss: 0.753854 	Validation Loss: 0.753281
Validation loss decreased (inf --> 0.753281).  Saving model ...
Epoch 2, Batch 1, loss: 0.793558 loss_age: 0.106713 loss_gender: 0.686845
Epoch 2, Batch 51, loss: 0.754631 loss_age: 0.053006 loss_gender: 0.692173
Epoch 2, Batch 101, loss: 0.752923 loss_age: 0.057439 loss_gender: 0.689309
Epoch 2, Batch 151, loss: 0.753510 loss_age: 0.078696 loss_gender: 0.695252
Epoch 2, Batch 201, loss: 0.753738 loss_age: 0.050277 loss_gender: 0.690306
Epoch 2, Batch 251, loss: 0.753836 loss_age: 0.071860 loss_gender: 0.691128
Epoch 2, Batch 301, loss: 0.754126 loss_age: 0.073477 loss_gender: 0.698241
Epoch 2, Batch 351, loss: 0.754405 loss_age: 0.058119 loss_gender: 0.691827
Epoch 2, Batch 401, loss: 0.754369 loss_age: 0.043771 loss_gender: 0.695935
Epoch 2, Batch 451, loss: 0.754333 loss_age: 0.091976 loss_gender: 0.702153
Epoch 2, Batch 501, loss: 0.754519 loss_age: 0.077331 loss_gender: 0.687849
Epoch 2, Batch 551, loss: 0.754139 loss_ag

# Load model

In [67]:
loaded_model = torch.load('model.pt')

In [87]:
def predict(image, max_age, device="cuda", gender_threshold=0.5):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = torch.from_numpy(image).permute(2, 0, 1).float()
    image = image.unsqueeze(0) 
    if device == "cuda":
      image = image.to(device)

    predicted = loaded_model(image)
    age = predicted["age"].item() * max_age
    prob_gender = predicted["gender"].item()
    gender = 1 if prob_gender > gender_threshold else 0
    
    return age, gender

In [89]:
image_path = "mega_age_gender/0_A14_G0.png"
image = cv2.imread(image_path)
pred_age, pred_gender = predict(image, 69, 'cuda')
print("pred_age: {}, pred_gender: {}".format(pred_age, pred_gender))

pred_age: 30.283694475889206, pred_gender: 1
